<a href="https://colab.research.google.com/github/arguntolga/ts_jupyter/blob/main/abovebelowma3_ERR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import yfinance as yf
import pandas as pd
import pandas.api.types as pdtypes
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Define your two stock tickers
#stock_list = ["EREGL.IS", "FROTO.IS", "THYAO.IS", "TTRAK.IS", "MIATK.IS"]
stock_list = [
    "AKBNK.IS", "ARCLK.IS", "ASELS.IS", "BIMAS.IS", "BRISA.IS", "ASTOR.IS",
    "DOHOL.IS", "SASA.IS", "EKGYO.IS", "ENKAI.IS", "EREGL.IS", "FROTO.IS",
    "GARAN.IS", "GUBRF.IS", "HALKB.IS", "HEKTS.IS", "ISCTR.IS", "KCHOL.IS",
    "KOZAA.IS", "KOZAL.IS", "KRDMD.IS", "MGROS.IS", "TTRAK.IS", "PETKM.IS",
    #"PGSUS.IS", "SAHOL.IS", "SISE.IS", "TAVHL.IS", "THYAO.IS", "TKFEN.IS"
    #"TTKOM.IS", "TUPRS.IS", "ULKER.IS", "VESTL.IS", "YKBNK.IS", "VESBE.IS",
    "ALFAS.IS", "TCELL.IS", "ALARK.IS", "BOBET.IS", "KONYA.IS", "AKSA.IS"
    #"AKSEN.IS", "YEOTK.IS", "GESAN.IS", "EUPWR.IS", "CWENE.IS", "YYLGD.IS"
]

# Set start and end dates
start_date = "2023-05-04"
end_date = "2024-05-31"

# Download data using yfinance
data = yf.download(stock_list, start=start_date, end=end_date)["Adj Close"]

print(data)

def calculate_moving_averages(stock_list, data, window_sizes):
  for stock in stock_list:
    for window in window_sizes:
      data[stock + '_MA_' + str(window)] = data[stock].rolling(window=window).mean()
  return data

def calculate_above_below_ma(stock_list, data, window_sizes):
  above_below_counts = {}
  for date in data.index:
    daily_counts = {}
    for window in window_sizes:
      above_count = 0
      for stock in stock_list:
        ma_col = f"{stock}_MA_{window}"
        above_count += (data.loc[date, stock] > data.loc[date, ma_col])
      below_count = len(stock_list) - above_count
      daily_counts["Above MA" + str(window)] = above_count
      daily_counts["Below MA" + str(window)] = below_count
      daily_counts["Ratio MA" + str(window)] = above_count / max(1,below_count)
    above_below_counts[date] = daily_counts
  return pd.DataFrame(above_below_counts).transpose()


def calculate_super_stock(stock_list, data, window_sizes):
  super_stock = 0
  super_stock_df = pd.DataFrame(index=data.index, columns=stock_list)
  for date in data.index:
    for stock in stock_list:
      super_stock_level = 0
      if data.loc[date, stock] > data.loc[date, f"{stock}_MA_{window_sizes[0]}"]:
          super_stock_level += 1
      for window_idx in range(len(window_sizes) - 1):
        current_ma_col = f"{stock}_MA_{window_sizes[window_idx]}"
        next_ma_col = f"{stock}_MA_{window_sizes[window_idx + 1]}"
        if data.loc[date, current_ma_col] > data.loc[date, next_ma_col]:
          super_stock_level += 1
      super_stock_df.loc[date, stock] = int(super_stock_level)
  return super_stock_df


# Define window sizes for moving averages
window_sizes = [10, 20, 50, 100, 200]

# Apply the moving average calculation
data_with_ma = calculate_moving_averages(stock_list, data.copy(), window_sizes)
display(data_with_ma)

def drop_oldest_rows(data, n_rows_to_drop):
  # Sort by index (assuming it's a date index)
  data_sorted = data.sort_index()
  # Select and drop the oldest N rows using iloc (integer location based indexing)
  data_dropped = data_sorted.iloc[n_rows_to_drop:]
  return data_dropped

data_dropped_ma = drop_oldest_rows(data_with_ma.copy(), 200)

# Calculate above/below MA counts
above_below_data = calculate_above_below_ma(stock_list, data_dropped_ma.copy(), window_sizes)
#print(above_below_data)

def green_red(value):
    if value < 0:
        return 'color: red; font-weight: bold'
    elif value > 1000:
        return 'color: green; font-weight: bold'
    else:
        return None

above_below_reversed = above_below_data.iloc[::-1]
#above_below_reversed.style.applymap(green_red)

def convert_to_int_if_zero_fraction(df):
  # Select numeric columns only (avoid errors with non-numeric data)
  numeric_cols = df.select_dtypes(include=[np.number])
  for col in numeric_cols:
    # Apply conversion function to the selected numeric column
    df[col] = df[col].apply(lambda x: int(x) if isinstance(x, (int, float)) and x % 1 == 0 else x)
  return df

# Apply conversion to all numeric columns
above_below_reversed = convert_to_int_if_zero_fraction(above_below_reversed.copy())  # Operate on a copy to avoid modifying original data
above_below_reversed.style.applymap(green_red)

#display(above_below_data.iloc[::-1])

#super_stock_data = calculate_super_stock(stock_list, data_dropped_ma, window_sizes)
#cm = sns.light_palette("green", as_cmap=True)
#super_stock_data.iloc[::-1].apply(pd.to_numeric).style.set_sticky(axis=1).background_gradient(cmap=cm)

[*********************100%%**********************]  30 of 30 completed


Ticker       AKBNK.IS     AKSA.IS    ALARK.IS   ALFAS.IS    ARCLK.IS  \
Date                                                                   
2023-05-04  14.892218   55.649345   53.731590  40.525002   90.730965   
2023-05-05  14.604723   55.316971   52.225552  36.474998   88.420906   
2023-05-08  15.802617   56.076687   54.168827  34.312500   90.534363   
2023-05-09  16.013447   54.129910   53.148613  31.875000   90.583519   
2023-05-10  16.109278   53.465157   50.233696  30.387501   90.976715   
...               ...         ...         ...        ...         ...   
2024-05-24  66.500000  122.000000  119.624115  78.500000  187.199997   
2024-05-27  68.000000  117.599998  117.045151  77.099998  185.899994   
2024-05-28  69.800003  119.500000  118.334633  76.099998  187.000000   
2024-05-29  66.000000  121.400002  118.731392  74.550003  184.000000   
2024-05-30  65.699997  121.300003  122.500000  74.650002  181.800003   

Ticker       ASELS.IS    ASTOR.IS    BIMAS.IS   BOBET.IS    BRI

Ticker,AKBNK.IS,AKSA.IS,ALARK.IS,ALFAS.IS,ARCLK.IS,ASELS.IS,ASTOR.IS,BIMAS.IS,BOBET.IS,BRISA.IS,...,KONYA.IS_MA_10,KONYA.IS_MA_20,KONYA.IS_MA_50,KONYA.IS_MA_100,KONYA.IS_MA_200,AKSA.IS_MA_10,AKSA.IS_MA_20,AKSA.IS_MA_50,AKSA.IS_MA_100,AKSA.IS_MA_200
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-04,14.892218,55.649345,53.731590,40.525002,90.730965,20.960352,45.200001,152.502319,20.200201,40.013248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-05,14.604723,55.316971,52.225552,36.474998,88.420906,20.161863,44.520000,153.578369,20.142761,38.456314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-08,15.802617,56.076687,54.168827,34.312500,90.534363,20.780691,44.000000,158.665039,19.191519,38.923393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-09,16.013447,54.129910,53.148613,31.875000,90.583519,19.982203,41.560001,161.893112,19.730883,38.067078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-10,16.109278,53.465157,50.233696,30.387501,90.976715,19.473166,40.860001,155.143494,19.338617,37.658386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,66.500000,122.000000,119.624115,78.500000,187.199997,59.250000,107.300003,467.500000,31.600000,119.500000,...,9475.25,10429.625,NaN,NaN,NaN,122.510001,120.080000,NaN,NaN,NaN
2024-05-27,68.000000,117.599998,117.045151,77.099998,185.899994,60.049999,103.900002,477.000000,30.420000,122.400002,...,9437.00,10322.250,NaN,NaN,NaN,122.380000,120.305000,NaN,NaN,NaN
2024-05-28,69.800003,119.500000,118.334633,76.099998,187.000000,59.599998,102.800003,481.750000,29.860001,118.400002,...,9373.50,10200.875,NaN,NaN,NaN,122.120000,120.455000,NaN,NaN,NaN


,Above MA10,Below MA10,Ratio MA10,Above MA20,Below MA20,Ratio MA20,Above MA50,Below MA50,Ratio MA50,Above MA100,Below MA100,Ratio MA100,Above MA200,Below MA200,Ratio MA200
2024-05-30 00:00:00,7,23,0.304348,15,15,1.000000,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-29 00:00:00,9,21,0.428571,16,14,1.142857,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-28 00:00:00,17,13,1.307692,19,11,1.727273,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-27 00:00:00,17,13,1.307692,18,12,1.500000,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-24 00:00:00,16,14,1.142857,19,11,1.727273,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-23 00:00:00,21,9,2.333333,22,8,2.750000,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-22 00:00:00,26,4,6.500000,24,6,4.000000,0,30,0.000000,0,30,0.000000,0,30,0.000000
2024-05-21 00:00:00,23,7,3.285714,23,7,3.285714,1,29,0.034483,1,29,0.034483,1,29,0.034483
2024-05-20 00:00:00,24,6,4.000000,23,7,3.285714,1,29,0.034483,1,29,0.034483,1,29,0.034483
2024-05-17 00:00:00,19,11,1.727273,22,8,2.750000,1,29,0.034483,1,29,0.034483,1,29,0.034483
